In [1]:
from sklearn.model_selection import train_test_split
from torchvision import datasets

data_dir = '../../dataset/skin_diseases/train'

full_dataset = datasets.ImageFolder(root=data_dir)

train_idx, test_idx = train_test_split(
    list(range(len(full_dataset))),
    test_size=0.2,
    random_state=42,
    stratify=full_dataset.targets,
)

In [2]:
from torch.utils.data import Subset

train_dataset = Subset(full_dataset, train_idx)
test_dataset = Subset(full_dataset, test_idx)

In [3]:
print(len(train_dataset))
print(len(test_dataset))

1195
299


In [4]:
image, label = train_dataset[0]
print(f'image : {image}, label : {label}')

image : <PIL.Image.Image image mode=RGB size=244x244 at 0x25027DFAB90>, label : 3


In [5]:
import matplotlib.pyplot as plt
import torch


def imshow(image, title):
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    image = image.permute(1, 2, 0)
    image = image * std + mean  # 역정규화
    # [0.485, 0.456, 0.406]
    # [0.229, 0.224, 0.225]
    plt.title(title)
    plt.imshow(image)  # permute 차원 순서 변경
    plt.show()

In [6]:
# import torchvision

# data_iter = iter(train_loader)
# images, labels = next(data_iter)

# out_images = torchvision.utils.make_grid(images)

# imshow(out_images, labels)

In [7]:
from torch import nn
from torchvision import models

from lib.model import create_model

weights = models.ResNet34_Weights.DEFAULT
rnet34_model = create_model(models.resnet34(weights=weights), name='ResNet34')
preprocess = weights.transforms()

print(f'평균(Mean): {preprocess.mean}')
print(f'표준편차(Std): {preprocess.std}')
print(f'권장 이미지 크기: {preprocess.resize_size}')

print(rnet34_model)
# for name, module in rnet34_model.named_parameters():
#     print(name, module.requires_grad)

for params in rnet34_model.parameters():
    params.requires_grad = False  # 가중치를 고정 frozen

rnet34_model.fc = nn.Linear(512, 5)  # 마지막 나올 fc만 수정 # 모델의 분류기만 바꿈
rnet34_model

평균(Mean): [0.485, 0.456, 0.406]
표준편차(Std): [0.229, 0.224, 0.225]
권장 이미지 크기: [256]
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, moment

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
from lib.dataset import create_loader, set_transform
from lib.train import eval_model, train_model
from lib.transform import create_transform

train_transform, test_transform = create_transform(
    (256, 256), preprocess.mean, preprocess.std
)

train_dataset_final, test_dataset_final = set_transform(
    (train_dataset, test_dataset), (train_transform, test_transform)
)

train_loader, test_loader = create_loader(train_dataset_final, test_dataset_final)

rnet34_model.train()
train_model(rnet34_model, train_loader, lr=1e-4, epochs=30)
eval_model(rnet34_model, test_loader, test_dataset)

c:\Workspaces\dl_practice\output\tensorboard\ResNet34\20260224


  0%|          | 0/38 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import torch.nn as nn
from torchvision import models

weights = models.ConvNeXt_Base_Weights.DEFAULT
conv_model = models.convnext_base(weights=weights)
conv_model.arc_name = 'convnext'

preprocess = weights.transforms()

print(f'평균(Mean): {preprocess.mean}')
print(f'표준편차(Std): {preprocess.std}')
print(f'권장 이미지 크기: {preprocess.resize_size}')

print(conv_model)

for name, module in conv_model.named_parameters():
    print(name, module.requires_grad)

for params in conv_model.parameters():
    params.requires_grad = False  # 가중치를 고정 frozen

conv_model.classifier[2] = nn.Linear(1024, 5)

print(conv_model.classifier)

In [ ]:
from lib.dataset import create_loader, set_transform
from lib.train import eval_model, train_model
from lib.transform import create_transform

train_loader, test_loader = create_loader(train_dataset, test_dataset, 32)

train_transform, test_transform = create_transform(
    (232, 232), preprocess.mean, preprocess.std
)

set_transform((train_dataset, test_dataset), (train_transform, test_transform))

train_model(conv_model, train_loader, 10)
eval_model(conv_model, test_loader, test_dataset)

In [ ]:
from torchvision import models

weights = models.EfficientNet_V2_S_Weights.DEFAULT
preprocess = weights.transforms()

eff_net_model = models.efficientnet_v2_s(weights=weights)
eff_net_model.arc_name = 'eff_net'

print(f'평균(Mean): {preprocess.mean}')
print(f'표준편차(Std): {preprocess.std}')
print(f'권장 이미지 크기: {preprocess.resize_size}')

eff_net_model

In [ ]:
import torch.nn as nn

for params in eff_net_model.parameters():
    params.requires_grad = False  # 가중치를 고정 frozen


eff_net_model.classifier[1] = nn.Linear(1280, 5)

print(eff_net_model.classifier)

In [ ]:
from lib.dataset import create_loader, set_transform
from lib.train import eval_model, train_model
from lib.transform import create_transform

train_loader, test_loader = create_loader(train_dataset, test_dataset, 32)

train_transform, test_transform = create_transform(
    (384, 384), preprocess.mean, preprocess.std
)

set_transform((train_dataset, test_dataset), (train_transform, test_transform))

train_model(eff_net_model, train_loader)
eval_model(eff_net_model, test_loader, test_dataset)

In [ ]:
import os

from lib.path import model_file_path


def save_checkpoint(ep, model, optim, loss):
    torch.save(
        {
            'epoch': ep,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': loss,
        },
        model_file_path(f'{model.arc_name}_checkpoint.pth'),
    )


def load_checkpoint(model, optim, device):
    path = model_file_path('checkpoint.pth')

    # 1. 파일 존재 여부 먼저 확인 (경로 오류 방지)
    if not os.path.exists(path):
        return 0
    try:
        # 2. weights_only=True로 로드 속도 및 안정성 향상 (PyTorch 최신버전 권장)
        checkpoint = torch.load(path, map_location=device)
        if checkpoint:
            # 3. 모델 가중치 로드
            model.load_state_dict(checkpoint['model_state_dict'])

            # 4. 옵티마이저 로드 (이 과정에서 메모리 점유가 늘어남)
            if 'optimizer_state_dict' in checkpoint:
                optim.load_state_dict(checkpoint['optimizer_state_dict'])

            # 5. 메모리 정리 (중요!)
            del checkpoint
            torch.cuda.empty_cache()  # GPU 사용 시

            return checkpoint.get('epoch', 0)

    except RuntimeError as e:
        print(f'체크포인트 로드 중 오류 발생: {e}')
        # 여기서 커널이 죽는다면 대부분 메모리 부족입니다.

    return 0


In [ ]:
from datetime import datetime

import tqdm
from torch import optim
from torch.utils.tensorboard import SummaryWriter

from lib.path import output_path

log_dir = output_path() / 'tensorboard' / datetime.now().strftime('%Y%m%d')
print(log_dir)
writer = SummaryWriter(log_dir=log_dir)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
optimizer = optim.Adam(eff_net_model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
epochs = 20
eff_net_model = eff_net_model.to(device)

start_ep = load_checkpoint(eff_net_model, optim, device)
start_ep = start_ep > 0 if start_ep else -1

count = 0

for ep in range(start_ep + 1, epochs):
    train_tqdm = tqdm.tqdm(train_loader)
    for image, labels in train_tqdm:
        optimizer.zero_grad()
        preds = eff_net_model(image.to(device))
        loss = criterion(preds, labels.to(device))
        writer.add_scalar('Loss/train', loss, count)
        count += 1
        loss.backward()
        optimizer.step()

        train_tqdm.set_description(f'epoch : {ep} loss : {loss.item():.2f}')

    save_checkpoint(ep, eff_net_model, optimizer, loss)

print('Complted')

In [ ]:
eff_net_model.eval()

with torch.no_grad():
    corrects = 0

    for image, labels in test_loader:
        preds = eff_net_model(image.to(device))
        pred = torch.max(preds, 1)[1]

        corrects += torch.sum(pred == labels.to(device).data)

        image_grid = torchvision.utils.make_grid(image)
        print(labels)
        imshow(image_grid.cpu(), title=pred)

    print(corrects, len(test_dataset))
    acc = corrects / len(test_dataset)
    print(f'정확도 : {acc}')
